In [1]:
import ee
import geemap
import datetime

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-ilyasnursyamsi05')
    print("Earth Engine initialized successfully.")
except ee.EEException as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Please ensure you have authenticated with 'ee.Authenticate()' and re-run.")
    exit()

# --- 1. CONFIGURATION (User-controlled variables) ---
ROI_PATH = "projects/ee-ilyasnursyamsi05/assets/CHC"
SEQ = ee.FeatureCollection(ROI_PATH)
start_year = 2022
end_year = 2024
COMMON_EXPORT_FOLDER = 'GEE_Exports_Weekly_OctNov15_24_CHC'

# --- Cloud Masking and Quality Band Functions ---

# Function for COPERNICUS/S2 and COPERNICUS/S2_SR (pre-harmonized)
def maskCloudsAndAddQuality_preHarmonized(image):
    has_qa60 = image.bandNames().contains('QA60')
    cloud_mask = ee.Algorithms.If(
        has_qa60,
        image.select('QA60').bitwiseAnd(1 << 10).eq(0).And(
            image.select('QA60').bitwiseAnd(1 << 11).eq(0)
        ),
        image.select('MSK_CLASSI_OPAQUE').eq(0).And(
            image.select('MSK_CLASSI_CIRRUS').eq(0)
        )
    )
    mask = ee.Image(cloud_mask)
    maskedImage = image.updateMask(mask)
    quality = mask.unmask(0).multiply(100).rename('quality')
    core_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    return maskedImage.select(core_bands).addBands(quality)

# Function for COPERNICUS/S2_SR_HARMONIZED (with GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED)
def maskCloudsAndAddQuality_harmonized(image):
    csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
    csFiltered = csPlus.filter(
        ee.Filter.eq('system:index', image.get('system:index'))
    )
    cloudMask = ee.Algorithms.If(
        csFiltered.size().gt(0),
        ee.Image(csFiltered.first()).select('cs_cdf').gte(0.70),
        ee.Image.constant(1)
    )
    maskedImage = image.updateMask(cloudMask)
    quality = ee.Image(cloudMask).unmask(0).multiply(100)
    core_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    return maskedImage.select(core_bands).addBands(quality.rename('quality'))

# --- Main processing loop for each year ---
for year in range(start_year, end_year + 1):
    print(f"\n--- Processing year: {year} ---\n")

    if year >= 2015 and year <= 2018:
        current_s2_collection = ee.ImageCollection("COPERNICUS/S2")
        current_masking_function = maskCloudsAndAddQuality_preHarmonized
        print(f"  Using COPERNICUS/S2 (Level-1C) for year {year} with adaptive QA60/MSK_CLASSI cloud mask.")
    elif year >= 2019 and year <= 2024:
        current_s2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        current_masking_function = maskCloudsAndAddQuality_harmonized
        print(f"  Using COPERNICUS/S2_SR_HARMONIZED (Level-2A) for year {year} with Cloud Score+ mask.")
    else:
        print(f"  No specific Sentinel-2 collection defined for year {year}, skipping.")
        continue

    full_year_start_date = ee.Date.fromYMD(year, 1, 1)
    full_year_end_date = full_year_start_date.advance(1, 'year')
    s2Filt_full_year = current_s2_collection.filterBounds(SEQ) \
        .filterDate(full_year_start_date, full_year_end_date) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50) \
        .map(current_masking_function)

    # Check if any images were found for the full year
    if s2Filt_full_year.size().getInfo() == 0:
        print(f"No Sentinel-2 images found for {year} in the full year range, skipping processing.")
        continue

    # --- Step 1: Create a collection of SEASONAL (3-month) quality mosaics ---
    seasonalComposites = ee.ImageCollection(
        ee.List.sequence(0, 3).map(lambda seasonOffset:
            ee.Algorithms.If(
                s2Filt_full_year.filterDate(
                    full_year_start_date.advance(ee.Number(seasonOffset).multiply(3), 'month'),
                    full_year_start_date.advance(ee.Number(seasonOffset).multiply(3).add(3), 'month')
                ).size().gt(0),
                s2Filt_full_year.filterDate(
                    full_year_start_date.advance(ee.Number(seasonOffset).multiply(3), 'month'),
                    full_year_start_date.advance(ee.Number(seasonOffset).multiply(3).add(3), 'month')
                ).qualityMosaic('quality').set('season_index', seasonOffset),
                ee.Image.constant(0).mask(ee.Image.constant(0)).set('season_index', seasonOffset)
            )
        )
    )

    # --- Step 2 & 3: Create and export WEEKLY composites for October and November ---
    print('Starting export tasks for this year. This may take a while...')
    
    months_to_process = [10, 11]

    for month_num in months_to_process:
        start_of_month = ee.Date.fromYMD(year, month_num, 1)
        end_of_month = start_of_month.advance(1, 'month')
        week_count = end_of_month.difference(start_of_month, 'week').ceil().getInfo()

        for week in range(week_count):
            start_of_week = start_of_month.advance(week, 'week')
            end_of_week = start_of_week.advance(1, 'week')

            weeklyCollection = current_s2_collection.filterBounds(SEQ) \
                                    .filterDate(start_of_week, end_of_week) \
                                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50) \
                                    .map(current_masking_function)
            
            # Determine the nearest seasonal index
            currentSeasonIndex = ee.Number(start_of_week.get('month')).divide(3).floor()
            
            # Select the appropriate seasonal background image
            seasonalBackground = ee.Image(seasonalComposites.filter(
                ee.Filter.eq('season_index', currentSeasonIndex)
            ).first())

            # Create the weekly composite, unmasking with the seasonal background
            composite_image = ee.Algorithms.If(
                weeklyCollection.size().gt(0),
                weeklyCollection.qualityMosaic('quality').unmask(seasonalBackground)
                    .set('system:time_start', start_of_week.millis())
                    .set('date_range', ee.Date(start_of_week).format('YYYYMMdd').cat('_').cat(ee.Date(end_of_week).advance(-1, 'day').format('YYYYMMdd')))
                    .set('week_num', week),
                ee.Image.constant(0).mask(ee.Image.constant(0)) \
                        .set('system:time_start', start_of_week.millis()) \
                        .set('date_range', ee.Date(start_of_week).format('YYYYMMdd').cat('_').cat(ee.Date(end_of_week).advance(-1, 'day').format('YYYYMMdd'))) \
                        .set('week_num', week)
            )
            
            # Perform the final band selection and type conversion
            image_to_export = ee.Image(composite_image).select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).toUint16()

            # Get properties for naming
            date_range_str = image_to_export.get('date_range').getInfo()

            # Define task description and file name prefix
            task_description = f'S2_Weekly_Composite_{year}_{date_range_str}'
            file_name_prefix = f'S2_Weekly_{date_range_str}'

            task = ee.batch.Export.image.toDrive(
                image=image_to_export,
                description=task_description,
                folder=COMMON_EXPORT_FOLDER,
                fileNamePrefix=file_name_prefix,
                scale=10,
                region=SEQ.geometry().bounds(),
                crs='EPSG:4326',
                maxPixels=1e13
            )
            task.start()
            print(f"Export task initiated for {task_description}")
            
    print(f"\nAll export tasks initiated for {year}. Check your Google Earth Engine Tasks tab.")

print('\nAll processing complete.')

Earth Engine initialized successfully.

--- Processing year: 2022 ---

  Using COPERNICUS/S2_SR_HARMONIZED (Level-2A) for year 2022 with Cloud Score+ mask.
Starting export tasks for this year. This may take a while...
Export task initiated for S2_Weekly_Composite_2022_20221001_20221007
Export task initiated for S2_Weekly_Composite_2022_20221008_20221014
Export task initiated for S2_Weekly_Composite_2022_20221015_20221021
Export task initiated for S2_Weekly_Composite_2022_20221022_20221028
Export task initiated for S2_Weekly_Composite_2022_20221029_20221104
Export task initiated for S2_Weekly_Composite_2022_20221101_20221107
Export task initiated for S2_Weekly_Composite_2022_20221108_20221114
Export task initiated for S2_Weekly_Composite_2022_20221115_20221121
Export task initiated for S2_Weekly_Composite_2022_20221122_20221128
Export task initiated for S2_Weekly_Composite_2022_20221129_20221205

All export tasks initiated for 2022. Check your Google Earth Engine Tasks tab.

--- Process